---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

--- 2025

# Manipulations simples d'une image avec OpenCV

Dans ce notebook, nous allons appliquer plusieurs transformations géométriques à une même image :

- **Resize** (redimensionnement)
- **Translation** (déplacement)
- **Rotation**
- **Crop** (rognage)
- **Zoom** (crop + agrandissement)
- **Flip** (symétrie miroir)

L'objectif est de voir comment on peut décrire ces transformations de façon mathématique (vecteurs, matrices) et comment elles sont mises en œuvre en pratique dans OpenCV.

## 1. Préparation de l'environnement
Si nécessaire, installe OpenCV avec `pip`. Dans un environnement déjà préparé (par exemple en local avec Anaconda ou un venv), cette étape peut être ignorée.

In [ ]:
%%bash
# À exécuter uniquement si OpenCV n'est pas installé
pip install opencv-python matplotlib

## 2. Chargement et affichage de l'image
On suppose que le fichier image s'appelle **`mon_image.jpg`** et se trouve dans le même dossier que ce notebook.

> ⚠ Si le nom ou le chemin du fichier est différent, pense à adapter la variable `image_path`.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Pour Jupyter : afficher les images directement dans le notebook
%matplotlib inline

image_path = 'mon_image.jpg'
img_bgr = cv2.imread(image_path)
if img_bgr is None:
    raise FileNotFoundError(f"Impossible de charger l'image : {image_path}")

# OpenCV charge en BGR, Matplotlib attend du RGB
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(4, 4))
plt.imshow(img_rgb)
plt.title('Image originale')
plt.axis('off')
plt.show()

print('Taille de l\'image (h, w, canaux) :', img_rgb.shape)

## 3. Resize (redimensionnement)
On modifie la taille de l'image, soit en fixant une nouvelle taille (largeur, hauteur), soit en utilisant des facteurs d'échelle `fx`, `fy`.

**Questions possibles :**
- Que se passe-t-il si on divise la taille par 2 ?
- Quelle est la nouvelle taille de l'image ?

In [ ]:
# Redimensionnement à une taille fixe (par exemple 300x300)
img_resize_fixed = cv2.resize(img_rgb, (300, 300))  # (width, height)

plt.figure(figsize=(4, 4))
plt.imshow(img_resize_fixed)
plt.title('Resize 300 x 300')
plt.axis('off')
plt.show()

# Redimensionnement par un facteur (par exemple 0.5 = réduction de moitié)
img_resize_scale = cv2.resize(img_rgb, None, fx=0.5, fy=0.5)

plt.figure(figsize=(4, 4))
plt.imshow(img_resize_scale)
plt.title('Resize par facteur 0.5')
plt.axis('off')
plt.show()

print('Taille originale :', img_rgb.shape)
print('Taille 300x300 :', img_resize_fixed.shape)
print('Taille facteur 0.5 :', img_resize_scale.shape)

## 4. Translation (déplacement de l'image)
On applique une **translation** à l'image, décrite par un vecteur $(t_x, t_y)$ :
\[(x', y') = (x + t_x, y + t_y)\]

En OpenCV, on utilise une matrice de transformation affine 2x3 :
\[ M = \begin{pmatrix} 1 & 0 & t_x \\ 0 & 1 & t_y \end{pmatrix} \]

**Question possible :**
- Que se passe-t-il si `t_x` est positif ou négatif ? et `t_y` ?

In [ ]:
h, w = img_rgb.shape[:2]
tx, ty = 50, 30  # déplacement de 50 px vers la droite, 30 px vers le bas
M_translation = np.float32([[1, 0, tx],
                            [0, 1, ty]])

img_translation = cv2.warpAffine(img_rgb, M_translation, (w, h))

plt.figure(figsize=(4, 4))
plt.imshow(img_translation)
plt.title(f'Translation (tx={tx}, ty={ty})')
plt.axis('off')
plt.show()

## 5. Rotation autour du centre de l'image
On effectue une rotation d'angle $\theta$ autour du centre de l'image. OpenCV fournit directement une fonction utilitaire pour calculer la matrice de rotation affine : `cv2.getRotationMatrix2D`.

**Question possible :**
- Pourquoi certains pixels "sortent" du cadre après rotation ?

In [ ]:
h, w = img_rgb.shape[:2]
center = (w // 2, h // 2)
angle = 30  # en degrés (sens anti-horaire)
scale = 1.0

M_rotation = cv2.getRotationMatrix2D(center, angle, scale)
img_rotation = cv2.warpAffine(img_rgb, M_rotation, (w, h))

plt.figure(figsize=(4, 4))
plt.imshow(img_rotation)
plt.title(f'Rotation de {angle}°')
plt.axis('off')
plt.show()

## 6. Crop (rognage)
Le **crop** correspond à la sélection d'un sous-rectangle de l'image. En termes de matrices, on prend simplement un sous-ensemble des lignes et colonnes.

**Question possible :**
- Comment choisir les indices pour recadrer une zone centrale ?

In [ ]:
# Exemple : crop d'une zone centrale
h, w = img_rgb.shape[:2]
y1, y2 = h // 4, 3 * h // 4
x1, x2 = w // 4, 3 * w // 4

crop = img_rgb[y1:y2, x1:x2]

plt.figure(figsize=(4, 4))
plt.imshow(crop)
plt.title('Crop (zone centrale)')
plt.axis('off')
plt.show()

print('Taille du crop :', crop.shape)

## 7. Zoom (crop + agrandissement)
Un **zoom numérique** classique consiste à :
1. **Croper** une petite zone de l'image.
2. **Agrandir** cette zone avec un `resize`.

C'est ce que fait un smartphone quand on "zoome" sur une photo après la prise.

**Question possible :**
- Que remarquez-vous sur la netteté de l'image après un zoom important ?

In [ ]:
# On repart du crop précédent et on l'agrandit
zoom_factor = 2.0
zoom = cv2.resize(crop, None, fx=zoom_factor, fy=zoom_factor)

plt.figure(figsize=(4, 4))
plt.imshow(zoom)
plt.title(f'Zoom x{zoom_factor} (crop puis resize)')
plt.axis('off')
plt.show()

print('Taille du crop :', crop.shape)
print('Taille après zoom :', zoom.shape)

## 8. Flip (symétrie miroir)
Le **flip** applique une symétrie :
- horizontale (gauche ↔ droite),
- verticale (haut ↔ bas),
- ou les deux.

C'est très utilisé en **data augmentation** pour entraîner des modèles d'IA sur des images variées.

**Question possible :**
- Que devient une image contenant du texte lorsqu'on applique un flip horizontal ?

In [ ]:
# Flip horizontal (miroir gauche-droite)
img_flip_h = cv2.flip(img_rgb, 1)

# Flip vertical (miroir haut-bas)
img_flip_v = cv2.flip(img_rgb, 0)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(img_flip_h)
plt.title('Flip horizontal')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_flip_v)
plt.title('Flip vertical')
plt.axis('off')
plt.tight_layout()
plt.show()

## 9. Conclusion
Dans ce notebook, nous avons appliqué plusieurs transformations simples à une image :

- **Resize** : changer la taille (échelle).
- **Translation** : déplacer l'image selon un vecteur $(t_x, t_y)$.
- **Rotation** : faire tourner l'image autour de son centre.
- **Crop** : extraire une sous-partie rectangulaire.
- **Zoom** : crop + agrandissement.
- **Flip** : symétrie miroir.

Ces opérations sont au cœur de nombreuses applications en **vision par ordinateur** et en **intelligence artificielle** (prétraitement, data augmentation, etc.).

---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

--- 2025